In [1]:
import pandas as pd
import csv
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.common.exceptions import NoSuchElementException

In [3]:
url_wsj = 'https://www.wsj.com'
url_login = 'https://sso.accounts.dowjones.com/login-page?op=localop&scope=openid%20idp_id%20roles%20email%20given_name%20family_name%20djid%20djUsername%20djStatus%20trackid%20tags%20prts%20suuid%20updated_at&client_id=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO&response_type=code&redirect_uri=https%3A%2F%2Faccounts.wsj.com%2Fauth%2Fsso%2Flogin&nonce=d45e5dea-7618-4439-96b8-6f7e3820e3cf&ui_locales=en-us-x-wsj-223-2&mars=-1&ns=prod%2Faccounts-wsj&state=nzJlrhAYPwjSuRAC.Cho3tHRv73tBiQjuVso8FzQsk6-ABjsZW4R3xzZSMTI&protocol=oauth2&client=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO#!/signin'
url_wsj_base_url = 'https://www.wsj.com/news/archive/2022/'
url_market = 'https://www.wsj.com/markets'
url_tech = 'https://www.wsj.com/technology'
csv_file = open('./articles4.csv', 'w')
csv_writer = csv.writer(csv_file)
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November" , "December"]


In [4]:
options = webdriver.ChromeOptions() #newly added 
options.headless = True #newly added
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = options)
#driver = webdriver.PhantomJS()
driver.get(url_login)
#sign_in_link = driver.find_element(By.LINK_TEXT,"Sign In")
#sigh_in_link = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.LINK_TEXT, "Sign in")))
#sign_in_link.click()
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "username")))
username.send_keys("login_email")
submit_button = driver.find_element(By.XPATH, ".//*[@id=\"basic-login\"]/div[1]/form/div[2]/div[6]/div[1]/button[2]")
submit_button.click()
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "password-login-password")))
password.send_keys("password")
sign_in_button = driver.find_element(By.XPATH, ".//*[@id=\"password-login\"]/div/form/div/div[5]/div[1]/button")
sign_in_button.click()
article_names = []

/var/folders/fl/gyy67ty525q7sz7xqdtgdcjr0000gn/T/ipykernel_24964/1315517480.py:2: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True #newly added


In [5]:
def get_link_from_webelem(lst):
    ret = []
    for l in lst:
        ret.append(l.get_attribute('href'))
        
    return ret

In [6]:
def scrape_articles():
    i = 0
    article_lst_uncleaned = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located(
                    (By.XPATH, "//h2[@class='WSJTheme--headline--unZqjb45 reset WSJTheme--heading-standard-s--2eMU4jl4 WSJTheme--standard--2eOdD903 typography--serif-display--ZXeuhS5E WSJTheme--small--2f09SjbK ']//a[@href]")
                )
            )
    article_lst = get_link_from_webelem(article_lst_uncleaned)
    for article in article_lst:
        
        driver.get(article)
        #article_lst2 = driver.find_elements(By.XPATH,
                        #"//h2[@class='WSJTheme--headline--unZqjb45 reset WSJTheme--heading-standard-s--2eMU4jl4 WSJTheme--standard--2eOdD903 typography--serif-display--ZXeuhS5E WSJTheme--small--2f09SjbK ']//a[@href]")
        
        #article_lst2 = WebDriverWait(driver, 5).until(
                #EC.presence_of_all_elements_located(
                    #(By.XPATH, "//h2[@class='WSJTheme--headline--unZqjb45 reset WSJTheme--heading-standard-s--2eMU4jl4 WSJTheme--standard--2eOdD903 typography--serif-display--ZXeuhS5E WSJTheme--small--2f09SjbK ']//a[@href]")
               # )
            #)
            #driver.execute_script("arguments[0].click();", article_lst2[i])
        #WebDriverWait(driver, 20).until(EC.element_to_be_clickable(article_lst2[i])).click()
            
        # only store articles that have a ticker in it's article content
        
       
            #articles that have tickers have a specific href
        
        try:
            ticker_lst_uncleaned = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.XPATH, "//*[starts-with(@href, '/market-data/quotes/') and contains(@href, 'mod=chiclets')]")))
            #ticker_lst_uncleaned = driver.find_elements(By.XPATH, "//*[starts-with(@href, '/market-data/quotes/') and contains(@href, 'mod=chiclets')]")
        except:
            continue
        ticker_cleaned = []
            #have to execute this weird line because text of inner child nodes are also displayed and we dont want that
        for ticker in ticker_lst_uncleaned:
            ticker_cleaned.append(driver.execute_script('return arguments[0].firstChild.textContent;', ticker).strip())
        
            
    
        #try to get article title
        try:
            #standard article name
            name = driver.find_element(By.XPATH, "//h1[@class='css-1lvqw7f-StyledHeadline e1ipbpvp0']").text
        except NoSuchElementException:
                #fancy article names (inside of a photo or something)
            name = driver.find_element(By.XPATH, "//h1[@class='evxsf2b0 css-456klt-StyledHeadline-BigTopHeadline-BigTopHeadline e1ipbpvp0']").text
        except:
            continue
            
        # if we have already seen this article then don't compute
        print(name)
        
        date = driver.find_element(By.NAME, "article.published").get_attribute("content")
        
            #article content
        #article_content = ""
        #text1 = driver.find_elements(By.XPATH,"//p[@class='css-xbvutc-Paragraph e3t0jlg0']")
        #for ele in text1:
            #article_content += ele.text
            
            # tell the chrome page to go back
        driver.back()
            # must sleep because page doesn't load in time
        article_dict = {
                        "article_headline": name,
                        "article_published_date": date,
                        "ticker_list": ticker_cleaned,
                        }
        
            # write row to csv
        csv_writer.writerow(article_dict.values())
        i += 1
        
    

In [7]:
def scrape_daylinks(url):
    driver.get(url)
    daylinks_lst_webelem = driver.find_elements(By.CLASS_NAME,
                        "WSJTheme--day-link--19pByDpZ "
                    )
    #daylinks_lst = driver.find_elements(By.XPATH,
                        #'//a[@class="WSJTheme--day-link--19pByDpZ "][@href]'
                    #)
    daylinks_lst = get_link_from_webelem(daylinks_lst_webelem)
    for daylink in daylinks_lst:
        driver.get(daylink)
        # have to recompute list because it goes boom upon page refresh
        #try:
            #daylink_lst2 =  WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located(
                    #(By.XPATH,
                        #'//a[@class="WSJTheme--day-link--19pByDpZ "][@href]'
                    #)
                #)
            #)
        
        
            #driver.execute_script("arguments[0].click();", daylink_lst2[i])
        #except:
            #driver.back()
            #i+=1
            #continue
        #daylink_lst2[i].click()
        scrape_articles()
        
        

In [8]:
for month in months:
    scrape_daylinks(url_wsj_base_url + month)

Starbucks Sales Rise as Cost Pressures Increase at the Coffee Chain
Google’s Grown-Up Phase Won’t Be Boring
GM Earnings Rose Sharply in 2021


ProtocolError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [9]:
csv_file.close()